# Spark Database Workloads
Tested only with __postgresql__

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


## Instantiate Classes

In [33]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('rezaware/')[0])
from rezaware.modules.etl.loader import sparkRDBM as db

''' restart initiate classes '''
if debug:
    import importlib
    db = importlib.reload(db)

__desc__ = "read and write files from and to postgresql database"
clsSDB = db.dataWorkLoads(
    desc=__desc__,
    db_type = 'PostgreSQL',
    db_driver=None,
    db_hostIP=None,
    db_port = None,
    db_name = 'property',
    db_schema='curated',
    db_user = 'rezaware',
    db_pswd = 'rezHERO',
    spark_partitions=None,
    spark_format = 'jdbc',
    spark_save_mode=None,
    spark_jar_dir = None,
)
if clsSDB.session:
    clsSDB._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional SPARKRDBM-libraries in LOADER-package of ETL-module imported successfully!

read and write files from and to postgresql database class initialization and load complete!


## List database tables

In [3]:
from pyspark.sql import functions as F
# /optiions ={}
tbl_sdf_ = clsSDB.get_db_table_info()
tbl_sdf_.select(F.col('table_schema'),F.col('table_name')).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `table_schema` cannot be resolved. Did you mean one of the following? [`table_name`, `table_type`, `table_catalog`, `is_typed`, `commit_action`].;
'Project ['table_schema, table_name#24]
+- Filter schema_names#37 IN (curated)
   +- Project [table_catalog#0, table_schema#1 AS schema_names#37, table_name#24, table_type#3, self_referencing_column_name#4, reference_generation#5, user_defined_type_catalog#6, user_defined_type_schema#7, user_defined_type_name#8, is_insertable_into#9, is_typed#10, commit_action#11]
      +- Project [table_catalog#0, table_schema#1, table_name#2 AS table_name#24, table_type#3, self_referencing_column_name#4, reference_generation#5, user_defined_type_catalog#6, user_defined_type_schema#7, user_defined_type_name#8, is_insertable_into#9, is_typed#10, commit_action#11]
         +- Relation [table_catalog#0,table_schema#1,table_name#2,table_type#3,self_referencing_column

## Load data from DB using SQL query

In [4]:
_from_date = '2023-04-24'
_to_date = '2023-04-29'

hotels_sdf_=clsSDB.read_data_from_table(
    select=None,
    db_table="util_refer",
    db_column="",
    lower_bound=None,
    upper_bound=None,
#     **_kwargs
)

if hotels_sdf_.count() > 0:
    print(hotels_sdf_.show(n=3,vertical=True))
else:
    print("Returned empty data set")

-RECORD 0-----------------------------
 ref_pk        | 1                    
 entity        | hotel_group          
 category      | hotel group category 
 value         | Beach Hotel and Spa  
 code          | HR                   
 description   | null                 
 source_uuid   | 3008365              
 data_source   | www.delta-check.c... 
 data_owner    | Delta Check          
 created_dt    | 2023-09-08 14:33:... 
 created_by    | rezaware             
 created_proc  | rezaware_io_impor... 
 modified_dt   | null                 
 modified_by   | null                 
 modified_proc | null                 
 deactivate_dt | null                 
-RECORD 1-----------------------------
 ref_pk        | 2                    
 entity        | hotel_group          
 category      | hotel group category 
 value         | Beach Resort         
 code          | R                    
 description   | null                 
 source_uuid   | 8008485              
 data_source   | https://

## Replace modify attributes with Nulls
* Set modify_proc & modify_by values to Null
* Drop the modify_dt column

In [7]:
from datetime import date, datetime, timedelta
from pyspark.sql import functions as F

_upsert_sdf = hotels_sdf_ \
                    .filter(F.col('ref_pk').isin([23])) \
                    .withColumn('deactivate_dt',F.lit(datetime.now())) \
#                     .withColumn('modified_proc',F.lit(None))
# _upsert_sdf = _upsert_sdf.drop(F.col('modified_dt'))
_upsert_sdf.show(n=3,vertical=True)

-RECORD 0-----------------------------
 ref_pk        | 23                   
 entity        | hotel_group          
 category      | hotel group category 
 value         | Urlaubsbauernhof     
 code          | UB                   
 description   | null                 
 source_uuid   | 8619052              
 data_source   | https://teams.mic... 
 data_owner    | Delta Check          
 created_dt    | 2023-09-08 14:33:... 
 created_by    | rezaware             
 created_proc  | rezaware_io_impor... 
 modified_dt   | null                 
 modified_by   | null                 
 modified_proc | null                 
 deactivate_dt | 2023-09-08 16:53:... 



## Upsert table to verify Modify values are auto added]
* Load data again to verify that modify_dt is added and modify_by & modify_proc have values

In [9]:
_db_name ='property'
_tbl_name='util_refer'
_pk = ['ref_pk']
_cols_not_for_update = ['created_dt','created_by','created_proc']
_options={
    "BATCHSIZE":1000,   # batch size to partition the dtaframe
    "PARTITIONS":1,    # number of parallel clusters to run
    "OMITCOLS":_cols_not_for_update,    # columns to be excluded from update
    
}
_records=clsSDB.upsert_sdf_to_table(
    save_sdf=_upsert_sdf,
    db_name =_db_name,
    db_table=_tbl_name,
    unique_keys=_pk,
    **_options,
)

print("Upserted %d records" % _records)

Wait a moment, writing data to postgresql property database ...
 INSERT INTO curated.util_refer (ref_pk, entity, category, value, code, description, source_uuid, data_source, data_owner, created_dt, created_by, created_proc, modified_dt, modified_by, modified_proc, deactivate_dt) VALUES %s  ON CONFLICT (ref_pk) DO UPDATE SET (entity, category, value, code, description, source_uuid, data_source, data_owner, modified_dt, modified_by, modified_proc, deactivate_dt) = (EXCLUDED.entity, EXCLUDED.category, EXCLUDED.value, EXCLUDED.code, EXCLUDED.description, EXCLUDED.source_uuid, EXCLUDED.data_source, EXCLUDED.data_owner, EXCLUDED.modified_dt, EXCLUDED.modified_by, EXCLUDED.modified_proc, EXCLUDED.deactivate_dt) ;


dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>
dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>


Upserted 1 records


dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>


## Get table next PK

In [16]:
next_pk = clsSDB.get_table_pk_nextval(
    tbl_name='hotel_detail',
    pk_attr ='hotel_pk',
)

next_pk.show()

+-------+
|nextval|
+-------+
|      7|
+-------+



## Set table PK index with new value

In [37]:
set_pk_val_ = clsSDB.set_table_pk_lastval(
    tbl_name= "prop_grp",
    pk_attr = "prop_grp_pk",
    set_val = 1
)
set_pk_val_

1